In [ ]:
#Importing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
from matplotlib import patches as patches
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from darksky import forecast
from datetime import date, timedelta, datetime
import sqlite3
import os
print(os.listdir("../input"))

In [ ]:
#Inputting the start and end dates to obtain weather data from
#There's a limit to call the API for a certain amount each day, so gathering the data took a few days dating back to 2016
day_delta = timedelta(days=1)
start_date = datetime(2017, 12, 31)
end_date = datetime(2018, 1, 2)

#After analyzing what the API gives, a dataframe is going to be filled with the following parameters:
daily_weather = pd.DataFrame(
{'time': [],
 'realtime': [],
 'summary': [],
 'icon': [],
 'precipIntensity': [],
 'precipProbability': [],
 'precipType': [],
 'temperature': [],
 'apparentTemperature': [],
 'dewPoint': [],
 'humidity': [],
 'pressure': [],
 'windSpeed': [],
 'windGust': [],
 'windBearing': [],
 'cloudCover': [],
 'uvIndex': [],
 'visibility': []})

In [ ]:
#To fill the daily_weather dataframe, the API has to be called for each historical hour
#Then store those values in the corresponding hour and repeat

for day in range((end_date - start_date).days):
    #Coordinates to call the API
    VICTORIA = 48.407326, -123.329773
    #The API requires the datetime to be in UNIX timecode
    dt = (start_date + day*day_delta).isoformat()
    weather = forecast('b03d590960edcd85cc8552dc812963de', *VICTORIA, time = dt)
    #The API defaults to imperial units, so we need to change them to SI units
    weather.refresh(units='si')
    #The below loop runs through each hour of the day before moving onto the next
    for hour in weather['hourly']['data']:
        daily_weather = daily_weather.append(
        {'time': hour.get('hour', 0),
        'realtime': datetime.fromtimestamp(hour['time']),
        'summary': hour.get('summary', 0),
        'icon': hour.get('icon', 0),
        'precipIntensity': hour.get('precipIntensity', 1),
        'precipProbability': hour.get('precipProbability', 1),
        'precipType': hour.get('precipType', 0),
        'temperature': hour.get('temperature', 0),
        'apparentTemperature': hour.get('apparentTemperature', 0),
        'dewPoint': hour.get('dewPoint', 0),
        'humidity': hour.get('humidity', 0),
        'pressure': hour.get('pressure', 0),
        'windSpeed': hour.get('windSpeed', 0),
        'windBearing': hour.get('windBearing', 0),
        'cloudCover': hour.get('cloudCover', 0),
        'uvIndex': hour.get('uvIndex', 0),
        'visibility': hour.get('visibility', 0),
        }, ignore_index=True)

In [ ]:
daily_weather

In [ ]:
#Working in Kaggle, the below code is used to export the dataframe to a CSV
#This was done 5 times to gather all the historical weather data as the DarkSkyAPI could only be called a limited number of times per day
from IPython.display import HTML
import base64

def create_download_link( daily_weather, title = "Download CSV file", filename = "daily_weather.csv"):  
    csv = daily_weather.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(daily_weather)

In [ ]:
fig = plt.figure(figsize = (10,10))
ax = fig.add_subplot(1, 1, 1)

# Major ticks every 20, minor ticks every 5
major_ticks = np.arange(0, 1001, 7)
minor_ticks = np.arange(0, 1001, 24)

ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)

# And a corresponding grid
ax.grid(which='both')

# Or if you want different settings for the grids:
ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)

plt.plot(daily_weather['temperature'])
plt.show()